# LLM Optimization Modelling Experiment

In [1]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [45]:
problem = '''We are delighted to welcome you, our newest intern on the Analytics team of Massachusetts General Hospital! You have been placed in a challenging role where you will be tasked with solving a real-world problem in the field of medical physics. We are building a pilot program in Boston, and if successful, your work could be applied widely in hospitals with limited capacity in many countries.

You are responsible for determining the best treatment plan for 17 patients who require radiotherapy. Your goal is to optimize the use of two possible treatments: photon therapy and proton therapy. While proton therapy is known to target tumors more precisely, it is also more expensive and has limited capacity in many countries. Therefore, you will need to balance the benefits of proton therapy with its limitations and cost to create an effective treatment plan for each patient.

To determine the best course of action for each patient, you will use a scoring system called the Biological Equivalent Dose (BED). This system allows you to calculate the effectiveness of each patient’s treatment plan by considering the number of proton fractions that can be used while still achieving the highest possible BED.

We have n=17 patients who need radiotherapy. Each patient i needs 15 fractions, which can be photon fractions, proton fractions, or a mix of photon and proton fractions (e.g. 4 proton fractions and 11 photon fractions). We want to use the limited proton therapy capacity as best as possible. We can calculate the BED score for each patient when p proton fractions and 15-p photon fractions are used, as BEDi(p,15-p), i.e., the BED when p proton and 15-p photon fractions are delivered for patient i. The higher the score, the better. 

The data file "ProblemData.csv" contains a 2D matrix of BED scores. It does not have an index. It was made in Excel and saved as csv. The columns are the number of proton fractions and each row represents a patient. In particular, the number at the (i,j) position is the score for patient i receiving j proton fractions. 

Suppose that the total maximal capacity C is 100 proton fractions. To maximize the total BED scores for all the patients, which patients should get proton fractions, and how many should they get? Formulate an integer linear optimization model to solve this problem. Assume you know the value BEDi(j,15-j) for each patient i. '''

## 2. Ask for parameters

In [48]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [49]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Variables:

* **x<sub>ij</sub>:** Binary variable equal to 1 if patient *i* receives *j* proton fractions, and 0 otherwise, for *i* = 1, 2, ..., 17 and *j* = 0, 1, 2, ..., 15. 


# 2. Ask for objective

In [50]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [51]:
Markdown(response2.text)

$$\max \sum_{i=1}^{17} \sum_{j=0}^{15} BED_{i}(j, 15-j) \cdot x_{ij}$$ 


# 3. Ask for constraints

In [52]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [53]:
Markdown(response3.text)

Subject to:

* **Capacity constraint:** The total number of proton fractions assigned to patients cannot exceed the available capacity, _C_.
  
  ∑<sub>i=1</sub><sup>17</sup> ∑<sub>j=0</sub><sup>15</sup> j * x<sub>ij</sub> ≤ C 

* **Single treatment per patient:** Each patient _i_ must receive a single treatment plan, meaning only one of  x<sub>ij</sub> can be equal to 1 for each _i_. 

  ∑<sub>j=0</sub><sup>15</sup> x<sub>ij</sub> = 1 for all i = 1,2,.., 17

* **Binary variable constraint:** The decision variables x<sub>ij</sub> are binary.

  x<sub>ij</sub> ∈ {0, 1} for all i = 1, 2, ..., 17 and j = 0, 1, 2, ..., 15 


## 3. Generate the pyomo code

In [62]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [63]:
#Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Sample Data - Replace with actual data
BED = {(i, j): i * j * (15 - j) / 100  # Sample function for BED, replace with actual calculation
       for i in range(1, 18) for j in range(0, 16)}
C = 100  # Sample capacity, replace with actual capacity

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.RangeSet(1, 17)  # Set of patients
model.J = pyo.RangeSet(0, 15)  # Set of proton fractions

# Variables
model.x = pyo.Var(model.I, model.J, within=pyo.Binary)  # Binary variable x_ij

# Objective function
def objective_rule(model):
    return sum(BED[i, j] * model.x[i, j] for i in model.I for j in model.J)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
def capacity_constraint(model):
    return sum(j * model.x[i, j] for i in model.I for j in model.J) <= C
model.capacity = pyo.Constraint(rule=capacity_constraint)

def single_treatment_constraint(model, i):
    return sum(model.x[i, j] for j in model.J) == 1
model.single_treatment = pyo.Constraint(model.I, rule=single_treatment_constraint)

# Solve
solver = pyo.SolverFactory('glpk')  # Choose a suitable solver
solver.solve(model)

# Print results
print("Optimal Solution:")
for i in model.I:
    for j in model.J:
        if pyo.value(model.x[i, j]) > 0.5:
            print(f"Patient {i} receives {j} proton fractions")
```

**Explanation:**

1. **Import pyomo:**  Import the Pyomo library.
2. **Sample Data:** I've included sample data for `BED` and `C`. You'll need to replace this with your actual data. 
3. **Model Initialization:** Create a concrete model object.
4. **Sets:** Define sets to represent patients (`I`) and proton fractions (`J`).
5. **Variables:** Create a binary variable `x` indexed over `I` and `J` representing the treatment plan.
6. **Objective Function:** Define the objective function to maximize the total `BED` based on the chosen treatment plans.
7. **Constraints:**
   - `capacity_constraint`: Ensures the total proton fractions assigned do not exceed the capacity `C`.
   - `single_treatment_constraint`:  Ensures each patient receives only one treatment plan.
8. **Solver:** Create a solver instance (here, using `glpk`) and solve the model.
9. **Results:** Print the optimal treatment plan for each patient. 


## 4. Input problem data and try running the generated code

In [75]:
import pyomo.environ as pyo
import pandas as pd

# Sample Data - Replace with actual data
# BED = {(i, j): i * j * (15 - j) / 100  # Sample function for BED, replace with actual calculation
#        for i in range(1, 18) for j in range(0, 16)}

#Data input done by human
df = pd.read_csv('ProblemData.csv', header=None)
BED = {}
for i, row in df.iterrows():
    for j, dose in enumerate(row):
        BED[(i+1, j)] = dose
#End

C = 100  # Sample capacity, replace with actual capacity

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.RangeSet(1, 17)  # Set of patients
model.J = pyo.RangeSet(0, 15)  # Set of proton fractions

# Variables
model.x = pyo.Var(model.I, model.J, within=pyo.Binary)  # Binary variable x_ij

# Objective function
def objective_rule(model):
    return sum(BED[i, j] * model.x[i, j] for i in model.I for j in model.J)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
def capacity_constraint(model):
    return sum(j * model.x[i, j] for i in model.I for j in model.J) <= C
model.capacity = pyo.Constraint(rule=capacity_constraint)

def single_treatment_constraint(model, i):
    return sum(model.x[i, j] for j in model.J) == 1
model.single_treatment = pyo.Constraint(model.I, rule=single_treatment_constraint)

# Solve
solver = pyo.SolverFactory('glpk')  # Choose a suitable solver
solver.solve(model)

# Print results
print("Optimal Solution:")
for i in model.I:
    for j in model.J:
        if pyo.value(model.x[i, j]) > 0.5:
            print(f"Patient {i} receives {j} proton fractions")
print(model.obj())

Optimal Solution:
Patient 1 receives 15 proton fractions
Patient 2 receives 8 proton fractions
Patient 3 receives 3 proton fractions
Patient 4 receives 0 proton fractions
Patient 5 receives 5 proton fractions
Patient 6 receives 0 proton fractions
Patient 7 receives 4 proton fractions
Patient 8 receives 13 proton fractions
Patient 9 receives 4 proton fractions
Patient 10 receives 5 proton fractions
Patient 11 receives 6 proton fractions
Patient 12 receives 0 proton fractions
Patient 13 receives 5 proton fractions
Patient 14 receives 0 proton fractions
Patient 15 receives 10 proton fractions
Patient 16 receives 10 proton fractions
Patient 17 receives 12 proton fractions
8.239999999999998


## 5. Correct the code to verify model viability (optional)

In [74]:
BED

{(1, 0): 0.14,
 (1, 1): 0.16,
 (1, 2): 0.18,
 (1, 3): 0.2,
 (1, 4): 0.22,
 (1, 5): 0.24,
 (1, 6): 0.26,
 (1, 7): 0.28,
 (1, 8): 0.3,
 (1, 9): 0.32,
 (1, 10): 0.34,
 (1, 11): 0.36,
 (1, 12): 0.38,
 (1, 13): 0.4,
 (1, 14): 0.42,
 (1, 15): 0.44,
 (2, 0): 0.2,
 (2, 1): 0.23,
 (2, 2): 0.27,
 (2, 3): 0.3,
 (2, 4): 0.35,
 (2, 5): 0.4,
 (2, 6): 0.44,
 (2, 7): 0.46,
 (2, 8): 0.5,
 (2, 9): 0.51,
 (2, 10): 0.52,
 (2, 11): 0.53,
 (2, 12): 0.53,
 (2, 13): 0.54,
 (2, 14): 0.54,
 (2, 15): 0.55,
 (3, 0): 0.5,
 (3, 1): 0.6,
 (3, 2): 0.7,
 (3, 3): 0.75,
 (3, 4): 0.75,
 (3, 5): 0.76,
 (3, 6): 0.76,
 (3, 7): 0.76,
 (3, 8): 0.77,
 (3, 9): 0.77,
 (3, 10): 0.77,
 (3, 11): 0.77,
 (3, 12): 0.78,
 (3, 13): 0.78,
 (3, 14): 0.78,
 (3, 15): 0.78,
 (4, 0): 0.3,
 (4, 1): 0.32,
 (4, 2): 0.34,
 (4, 3): 0.36,
 (4, 4): 0.38,
 (4, 5): 0.4,
 (4, 6): 0.42,
 (4, 7): 0.44,
 (4, 8): 0.46,
 (4, 9): 0.48,
 (4, 10): 0.5,
 (4, 11): 0.52,
 (4, 12): 0.54,
 (4, 13): 0.56,
 (4, 14): 0.58,
 (4, 15): 0.6,
 (5, 0): 0.01,
 (5, 1): 0.06,


## 6. Print the responses

In [65]:
response.text

'## Variables:\n\n* **x<sub>ij</sub>:** Binary variable equal to 1 if patient *i* receives *j* proton fractions, and 0 otherwise, for *i* = 1, 2, ..., 17 and *j* = 0, 1, 2, ..., 15. \n'

In [66]:
response2.text

'$$\\max \\sum_{i=1}^{17} \\sum_{j=0}^{15} BED_{i}(j, 15-j) \\cdot x_{ij}$$ \n'

In [67]:
response3.text

'Subject to:\n\n* **Capacity constraint:** The total number of proton fractions assigned to patients cannot exceed the available capacity, _C_.\n  \n  ∑<sub>i=1</sub><sup>17</sup> ∑<sub>j=0</sub><sup>15</sup> j * x<sub>ij</sub> ≤ C \n\n* **Single treatment per patient:** Each patient _i_ must receive a single treatment plan, meaning only one of  x<sub>ij</sub> can be equal to 1 for each _i_. \n\n  ∑<sub>j=0</sub><sup>15</sup> x<sub>ij</sub> = 1 for all i = 1,2,.., 17\n\n* **Binary variable constraint:** The decision variables x<sub>ij</sub> are binary.\n\n  x<sub>ij</sub> ∈ {0, 1} for all i = 1, 2, ..., 17 and j = 0, 1, 2, ..., 15 \n'

In [68]:
response4.text

'```python\nimport pyomo.environ as pyo\n\n# Sample Data - Replace with actual data\nBED = {(i, j): i * j * (15 - j) / 100  # Sample function for BED, replace with actual calculation\n       for i in range(1, 18) for j in range(0, 16)}\nC = 100  # Sample capacity, replace with actual capacity\n\n# Model\nmodel = pyo.ConcreteModel()\n\n# Sets\nmodel.I = pyo.RangeSet(1, 17)  # Set of patients\nmodel.J = pyo.RangeSet(0, 15)  # Set of proton fractions\n\n# Variables\nmodel.x = pyo.Var(model.I, model.J, within=pyo.Binary)  # Binary variable x_ij\n\n# Objective function\ndef objective_rule(model):\n    return sum(BED[i, j] * model.x[i, j] for i in model.I for j in model.J)\nmodel.obj = pyo.Objective(rule=objective_rule, sense=pyo.maximize)\n\n# Constraints\ndef capacity_constraint(model):\n    return sum(j * model.x[i, j] for i in model.I for j in model.J) <= C\nmodel.capacity = pyo.Constraint(rule=capacity_constraint)\n\ndef single_treatment_constraint(model, i):\n    return sum(model.x[i, 